In [1]:
import pandas as pd
import numpy as np
import psycopg2

from sqlalchemy import create_engine


In [2]:
raw_employees = pd.read_csv('/Users/shirleyxueyinghe/programming/barcelona_gse/databases/warehousing-operational-databases/homework/extracts/employees.csv')


raw_orders = pd.read_csv('/Users/shirleyxueyinghe/programming/barcelona_gse/databases/warehousing-operational-databases/homework/extracts/orders.csv')
raw_orders.rename(columns={'sales_rep_employee_number': 'employee_number'}, inplace=True)

raw_products = pd.read_csv('/Users/shirleyxueyinghe/programming/barcelona_gse/databases/warehousing-operational-databases/homework/extracts/products.csv')



In [3]:
overlap_cols = ['city', 'state', 'country']

def change_prefix(columns):
    for col in overlap_cols:
        raw_employees.rename(columns={col : 'employee_'+ col}, inplace=True)
        raw_orders.rename(columns={col: 'order_' + col}, inplace=True)
    return raw_employees, raw_orders

change_prefix(overlap_cols)    

(    office_code  employee_number  last_name first_name  reports_to  \
 0             1             1002     Murphy      Diane         NaN   
 1             1             1056  Patterson       Mary      1002.0   
 2             1             1076   Firrelli       Jeff      1002.0   
 3             1             1143        Bow    Anthony      1056.0   
 4             1             1165   Jennings     Leslie      1143.0   
 5             1             1166   Thompson     Leslie      1143.0   
 6             2             1188   Firrelli      Julie      1143.0   
 7             2             1216  Patterson      Steve      1143.0   
 8             3             1286      Tseng   Foon Yue      1143.0   
 9             3             1323     Vanauf     George      1143.0   
 10            4             1102     Bondur     Gerard      1056.0   
 11            4             1337     Bondur       Loui      1102.0   
 12            4             1370  Hernandez     Gerard      1102.0   
 13   

In [4]:
full_joined_table = raw_orders.join(raw_products.set_index('product_code'), on=['product_code'], how='outer')\
    .join(raw_employees.set_index('employee_number'), on='employee_number', how='outer')\
    
    

full_joined_table['order_date'] = pd.to_datetime(full_joined_table['order_date'])

# Making new columns I want in star table

In [5]:
full_joined_table['profit'] = full_joined_table['quantity_ordered'] * (full_joined_table['price_each'] - full_joined_table['buy_price'])
full_joined_table['total_sale_value'] = full_joined_table['quantity_ordered'] * full_joined_table['price_each']


In [6]:
full_joined_table['day_id'] = pd.to_datetime(full_joined_table['order_date']).dt.dayofweek
full_joined_table['quarter_id'] = pd.to_datetime(full_joined_table['order_date']).dt.quarter

# Building Dimension Tables

In [7]:
raw_employees.head()

,office_code,employee_number,last_name,first_name,reports_to,job_title,employee_city,employee_state,employee_country,office_location
0,1,1002,Murphy,Diane,NaN,President,San Francisco,CA,USA,b'\xe6\x10\x00\x00\x01\x01\x00\x00\x00\xd3\xda...
1,1,1056,Patterson,Mary,1002.0,VP Sales,San Francisco,CA,USA,b'\xe6\x10\x00\x00\x01\x01\x00\x00\x00\xd3\xda...
2,1,1076,Firrelli,Jeff,1002.0,VP Marketing,San Francisco,CA,USA,b'\xe6\x10\x00\x00\x01\x01\x00\x00\x00\xd3\xda...
3,1,1143,Bow,Anthony,1056.0,Sales Manager (NA),San Francisco,CA,USA,b'\xe6\x10\x00\x00\x01\x01\x00\x00\x00\xd3\xda...
4,1,1165,Jennings,Leslie,1143.0,Sales Rep,San Francisco,CA,USA,b'\xe6\x10\x00\x00\x01\x01\x00\x00\x00\xd3\xda...


In [8]:
def build_dimension(star, name, index, columns):
    dim_index = name
    dim = star[columns].dropna(how='all').drop_duplicates().reset_index().rename_axis(dim_index).drop('index', axis=1).reset_index()
    return dim


customer_dim = build_dimension(full_joined_table, 'customer_id', 'customer_number', ['customer_number', 'customer_name', 'contact_last_name', 'contact_first_name', 'credit_limit'])

product_dim = build_dimension(full_joined_table, 'product_id','product_code', ['product_code', 'product_name', 'product_scale', 'product_description', 'html_description'] )

city_dim = build_dimension(full_joined_table, 'city_id', 'order_city', ['order_city', 'order_state', 'order_country'])

employee_dim = build_dimension(full_joined_table, 'employee_id', 'employee_number', ['employee_number', 'last_name', 'first_name', 'reports_to', 'job_title', 'employee_city', 'employee_state', 'employee_country', 'office_code'])

product_line_dim = full_joined_table[['product_line', 'product_code']].groupby('product_line').count().reset_index()
product_line_dim.rename(columns={'product_code':'product_line_id'}, inplace=True)
product_line_dim['product_line_id']= [0,1,2,3,4,5,6]


In [9]:
day_of_week_dim = build_dimension(full_joined_table, 'day_of_week', 'day_id', ['day_id', 'order_date'])
day_of_week_dim.drop('day_of_week', axis=1, inplace=True)     
day_of_week_dim['order_date'] = pd.to_datetime(day_of_week_dim['order_date'])
day_of_week_dim['day'] = day_of_week_dim['order_date'].dt.day_name()

    
    
quarter_dim = build_dimension(full_joined_table, 'quarter', 'quarter_id', ['quarter_id', 'order_date'])
quarter_dim.drop('quarter', axis=1, inplace=True)
quarter_dim['order_date'] = pd.to_datetime(quarter_dim['order_date'])


# Final Star Table

In [10]:
city_dim.head()

,city_id,order_city,order_state,order_country
0,0,Nashua,NH,USA
1,1,Allentown,PA,USA
2,2,Cambridge,MA,USA
3,3,Brickhaven,MA,USA
4,4,New Bedford,MA,USA


In [11]:
full_joined_table = full_joined_table.merge(day_of_week_dim, on=['order_date','day_id'], how='outer')
full_joined_table = full_joined_table.merge(quarter_dim, on=['order_date', 'quarter_id'], how='outer')
full_joined_table = full_joined_table.merge(product_line_dim, on=['product_line'], how='outer')
full_joined_table = full_joined_table.merge(customer_dim, on=['customer_number'], how='outer')
full_joined_table = full_joined_table.merge(product_dim, on=['product_code'], how='outer')
full_joined_table = full_joined_table.merge(city_dim, on='order_city', how='outer')
full_joined_table = full_joined_table.merge(employee_dim, on='employee_number', how='outer')


In [13]:
cols_to_keep = ['customer_id',\
                'order_number',\
                'order_date',\
                'employee_id',\
                 'profit',\
                'total_sale_value',\
                'day_id',\
                'quarter_id',\
                'product_line_id', \
                'city_id', \
                'product_id'
               ]

In [15]:
final_star = full_joined_table[cols_to_keep].reset_index()
final_star['product_line_id'] = final_star['product_line_id']


In [16]:
# Checking for nulls and dropping
nulls_to_drop = list(final_star[final_star['profit'].isnull()].index)
nulls_to_drop

final_star.drop(nulls_to_drop, inplace=True)


# Pushing Data to Postgres

In [17]:
conn = psycopg2.connect("host=localhost dbname=shirley_homework user=postgres")
cur = conn.cursor()
engine = create_engine('postgresql://postgres: @localhost:5432/shirley_homework')

In [18]:
final_star.to_sql(name ='star', con=engine, index=False, if_exists='replace')
customer_dim.to_sql(name ='customer_dim', con=engine, index=False, if_exists='replace', index_label='customer_id')
product_dim.to_sql(name ='product_dim', con=engine, index=False, if_exists='replace', index_label='product_id')
product_line_dim.to_sql(name='product_line_dim', con=engine, index=False, if_exists='replace', index_label='product_line_id')
employee_dim.to_sql(name ='employee_dim', con=engine, index=False, if_exists='replace', index_label='employee_id')
city_dim.to_sql(name ='city_dim', con=engine, index=False, if_exists='replace', index_label='city_id')
day_of_week_dim.to_sql(name='day_of_week_dim', con=engine, index=False, if_exists='replace', index_label='day_id')
quarter_dim.to_sql(name='quarter_dim', con=engine, index=False, if_exists='replace', index_label='quarter_id')




## checking how things look


In [19]:
final_star.head()

,index,customer_id,order_number,order_date,employee_id,profit,total_sale_value,day_id,quarter_id,product_line_id,city_id,product_id
0,0,0.0,10100.0,2003-01-06,0.0,1479.00,4080.00,0.0,1.0,6.0,0.0,0.0
1,1,0.0,10100.0,2003-01-06,0.0,1089.50,2754.50,0.0,1.0,6.0,0.0,1.0
2,2,0.0,10100.0,2003-01-06,0.0,708.40,1660.12,0.0,1.0,6.0,0.0,2.0
3,3,0.0,10100.0,2003-01-06,0.0,663.46,1729.21,0.0,1.0,6.0,0.0,3.0
4,4,0.0,10322.0,2004-11-04,0.0,3114.50,6038.50,3.0,4.0,6.0,0.0,4.0


In [20]:
product_dim.head()

,product_id,product_code,product_name,product_scale,product_description,html_description
0,0,S18_1749,1917 Grand Touring Sedan,1:18,This 1:18 scale replica of the 1917 Grand Tour...,NaN
1,1,S18_2248,1911 Ford Town Car,1:18,"Features opening hood, opening doors, opening ...",NaN
2,2,S18_4409,1932 Alfa Romeo 8C2300 Spider Sport,1:18,This 1:18 scale precision die cast replica fea...,NaN
3,3,S24_3969,1936 Mercedes Benz 500k Roadster,1:24,This model features grille-mounted chrome horn...,NaN
4,4,S18_2325,1932 Model A Ford J-Coupe,1:18,This model features grille-mounted chrome horn...,NaN


In [21]:
customer_dim.head()

,customer_id,customer_number,customer_name,contact_last_name,contact_first_name,credit_limit
0,0,363.0,Online Diecast Creations Co.,Young,Dorothy,114200.0
1,1,157.0,Diecast Classics Inc.,Leong,Kelvin,100600.0
2,2,286.0,Marta's Replicas Co.,Hernandez,Marta,123700.0
3,3,198.0,Auto-Moto Classics Inc.,Taylor,Leslie,23000.0
4,4,462.0,FunGiftIdeas.com,Benitez,Violeta,85800.0


In [22]:
day_of_week_dim.head()

,day_id,order_date,day
0,0.0,2003-01-06,Monday
1,3.0,2004-11-04,Thursday
2,0.0,2005-05-30,Monday
3,3.0,2004-08-19,Thursday
4,2.0,2004-10-13,Wednesday


### looking at some queries I wrote in SQL through this notebook

In [23]:
#Get the top 3 product types that have proven most profitable

query = "SELECT p.product_line, SUM(profit) AS total_profit \
FROM star \
LEFT JOIN product_line_dim p \
USING(product_line_id) \
GROUP BY p.product_line \
LIMIT 3;"

print(pd.read_sql(query, engine))

  product_line  total_profit
0  Motorcycles     471459.83
1        Ships     263016.69
2       Planes     377613.18


In [24]:
#Get the top 3 products by most items sold
query = "SELECT prod_dim.product_name, COUNT(*) AS counts \
FROM star \
LEFT JOIN  product_dim as prod_dim \
USING (product_id) \
GROUP BY prod_dim.product_name \
ORDER BY counts DESC \
LIMIT 3;"

print(pd.read_sql(query, engine))

                     product_name  counts
0     1992 Ferrari 360 Spider red      53
1  America West Airlines B757-200      29
2                      HMS Bounty      29


In [25]:
#Get the most profitable day of the week
query = "SELECT d_dim.day, SUM(profit) AS total_profit \
FROM star \
JOIN day_of_week_dim as d_dim \
USING (day_id) \
GROUP BY d_dim.day \
ORDER BY total_profit DESC \
LIMIT 1;"

print(pd.read_sql(query, engine))



      day  total_profit
0  Friday   43963595.64
